## Checkpoint

In [1]:
import os
from glob import glob

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

import matplotlib.pyplot as plt

%matplotlib inline

## Hyper parameter Tunning

In [2]:
num_epochs = 10
batch_size = 32
learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net)
model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0     

## Data Process

In [4]:
## (1) train, test 데이터 읽어오기
print(os.listdir('../dataset/cifar/'))

train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

print(len(train_paths), len(test_paths))

## (2) label name 반환
def get_class_name(path):
    return path.split('_')[-1].replace('.png','')

## (3) label name 고유값 list - class_names
train_labels = [get_class_name(path) for path in train_paths] # 50000개
class_names = np.unique(train_labels) # 해당 class 개로 줄여짐

# (4) label 반환 onehot encoding 된 형태로 or argmax 로 해당 index
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) 

# (5) 최종 image, label 반환
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32)/.255 # rescale
    
    label = get_label(path)
    return image, label

# (6) image preprocess (랜덤 Generator)
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE # 작동하는 Network가 스스로 설정하고 Dataset을 잘 불러올 수 있게 결정하라는 것


# train dataset 뽑기
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

# test set 데이터 뽑기
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()
      

['labels.txt', 'test', 'train']
1000 1000


## Checkpoint

In [12]:
save_path = 'checkpoints'

# checkpoint 는 개선될 때마다 저장되는 것
# monitor => monitor 할 정보
# mode = 'max' (모니터 정보가 accuracy 일 경우) / mode='min' (모니터 정보가 loss 인 경우) 
# verbose = 1 이면 보여줌 0 이면 안보여줌
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, 
                                                monitor = 'val_acc', # accuracy를 monitor 해라 
                                                verbose=1,
                                                mode = 'max', # accuracy면 max, loss면 min
                                                save_best_only=True, # accuracy가 올라가면 저장하는 것
                                  )

## Training

In [13]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_steps = validation_steps,
    epochs = num_epochs,
    callbacks=[checkpoint]
)

Epoch 1/10
31/31 [==============================] - 6s 200ms/step - loss: 2.3028 - accuracy: 0.1095
Epoch 2/10
31/31 [==============================] - 8s 268ms/step - loss: 2.3021 - accuracy: 0.1167
Epoch 3/10
31/31 [==============================] - 7s 241ms/step - loss: 2.2967 - accuracy: 0.1179
Epoch 4/10
31/31 [==============================] - 6s 197ms/step - loss: 2.2968 - accuracy: 0.1303
Epoch 5/10
31/31 [==============================] - 4s 127ms/step - loss: 2.2918 - accuracy: 0.1260
Epoch 6/10
31/31 [==============================] - 4s 125ms/step - loss: 2.2965 - accuracy: 0.1147
Epoch 7/10
31/31 [==============================] - 4s 122ms/step - loss: 2.2919 - accuracy: 0.1219
Epoch 8/10
31/31 [==============================] - 4s 123ms/step - loss: 2.2895 - accuracy: 0.1341
Epoch 9/10
31/31 [==============================] - 4s 118ms/step - loss: 2.2981 - accuracy: 0.1282
Epoch 10/10
31/31 [==============================] - 4s 121ms/step - loss: 2.2911 - accuracy: 0.1331